In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive

# Connect Google Drive (make sure your model is saved in Google Drive)
drive.mount('/content/drive')

Mounted at /content/drive


**## Link to the model**
https://drive.google.com/file/d/1IOfYaFUjN1Gk1tn2ASye-CrggFDA4-zQ/view?usp=drive_link


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, Add, GlobalAveragePooling2D, Dense, Flatten
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.datasets import fashion_mnist
#import cv2

# Loading of Fashion Dataset

In [ ]:
fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
train_images.shape, train_labels.shape, test_images.shape,test_labels.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [ ]:
train_labels

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [ ]:
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
# # Define the Residual Block
# def residual_block(x, filters, kernel_size=3, stride=1):
#     shortcut = x

#     # First convolution layer
#     x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
#     x = BatchNormalization()(x)
#     x = ReLU()(x)

#     # Second convolution layer
#     x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
#     x = BatchNormalization()(x)

#     # Add the shortcut to the output
#     x = Add()([x, shortcut])
#     x = ReLU()(x)

#     return x

In [12]:
def residual_block(x, filters, kernel_size=3, stride=1, projection=True):
    # Shortcut
    shortcut = x

    # First convolution
    x = Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Second convolution
    x = Conv2D(filters, kernel_size=kernel_size, strides=1, padding='same')(x)
    x = BatchNormalization()(x)

    # Check if we need to project the shortcut (change dimensions)
    if projection:
        shortcut = Conv2D(filters, kernel_size=1, strides=stride, padding='same')(shortcut)
        shortcut = BatchNormalization()(shortcut)

    # Add the shortcut to the output
    x = Add()([x, shortcut])
    x = ReLU()(x)

    return x

In [13]:
# Build the ResNet-like model
def build_resnet(input_shape, num_classes):

    inputs = Input(shape=input_shape)

    # Initial convolution layer
    x = Conv2D(32, kernel_size=3, strides=1, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Residual Blocks
    x = residual_block(x, filters=32)
    x = residual_block(x, filters=32)
    x = residual_block(x, filters=32)

    x = residual_block(x, filters=64, stride=1)
    x = residual_block(x, filters=64)
    x = residual_block(x, filters=64)

    x = residual_block(x, filters=128, stride=1)
    x = residual_block(x, filters=128)
    x = residual_block(x, filters=128)

    x = GlobalAveragePooling2D()(x)

    # Add Flatten layer to convert to a 1D vector
    x = Flatten()(x)

    # Fully connected layer for classification
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, x)
    return model

In [14]:
# Create the ResNet model
resnet_model = build_resnet((28, 28, 1), 10)

In [15]:
# Print the model summary
resnet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 28, 28, 32)           320       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 28, 28, 32)           128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 re_lu (ReLU)                (None, 28, 28, 32)           0         ['batch_normalization[0][0

In [16]:
# Compile the model (customize optimizer, loss, and metrics as needed)
resnet_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model with your training data
history = resnet_model.fit(train_images, train_labels, epochs=10, validation_split=0.2,batch_size=32)

Epoch 1/10
 392/1500 [======>.......................] - ETA: 51:51 - loss: 0.7232 - accuracy: 0.7372

In [ ]:
# Plot the learning curve
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.title('Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.title('Accuracy Curve')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
# Save the model to a file
resnet_model.save('/content/drive/My Drive/MyDeepLearning/fashion_mnist_model.h5')

In [ ]:
# Load the saved model
loaded_model = keras.models.load_model('/content/drive/My Drive/MyDeepLearning/fashion_mnist_model.h5')

In [ ]:
# Evaluate the loaded model on the test data
test_loss, test_accuracy = loaded_model.evaluate(test_images, test_labels)
print("Test accuracy:", test_accuracy)

313/313 [==============================] - 231s 735ms/step - loss: 0.2485 - accuracy: 0.9114
Test accuracy: 0.9114000201225281
